# Using COVID19API and Alpha Vantage Data to create a market analysis

In [70]:
#Used for pulling the data
import urllib.request 
import json
#Used for formatting the data
import pandas as pd
#Used for data visualisation
import matplotlib.pyplot as plt
from matplotlib import style

In [71]:
#Returns a pandaDF of the closing stock prices in the last x amount of days
def getPriceOfStock(companySymbol, numOfDays): 
    url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol="+companySymbol+"&outputsize=full&apikey=Y11HUWNU7HM58RRL"
    json_obj = urllib.request.urlopen(url)
    #Finds the data from the url
    data = json.load(json_obj)
    #Choosing the revelant key from the data dictionary
    dates = data["Time Series (Daily)"] 
    #Initializing lists to store the data
    final_prices = [] 
    calendar = []
    counter = 0
    #For-loop used to run through every trading day 
    for day in dates: 
        counter += 1
        #Limits the amount of data using the number of days wanted
        if counter > numOfDays: 
            break
        #Gets the stock's price - This will include Open, High, Low, Closed prices
        prices = dates[day]
        #Choosing the stock's closed price
        priceNum = float(prices['4. close']) 
        final_prices.append(priceNum)
        calendar.append(day)
    #Dictionary containing the company and it's stock prices 
    final_prices.reverse()
    calendar.reverse()
    stockDict = {companySymbol : final_prices} #CHANGE
    #Formatting the pulled data to a Panda DataFrame
    df = pd.DataFrame(data = stockDict, index = pd.to_datetime(calendar))
    return df

In [72]:
#NOTE: Alpha Vantage only allows 5 API calls a minute, 500 a day -> Can only have an input size of list size 5
#Function to format (multiple) stocks into a PandaDF and alligns the dates
def bundleStockPrices(arrayOfCompanySymbols, numOfDays):
    largeTable = getPriceOfStock(arrayOfCompanySymbols[0], numOfDays) 
    del arrayOfCompanySymbols[0]
    for symbol in arrayOfCompanySymbols:
        smallTable = getPriceOfStock(symbol, numOfDays)
        #Concatenating the tables together -> will provide the intersection of all the data -> will provide <= 200 day data values
        #largeTable = pd.concat([largeTable, smallTable.reindex(largeTable.index)], axis = 1)
        largeTable = pd.concat([largeTable, smallTable], axis = 1, join ='inner')
    print("Any null data: "+str(largeTable.isnull().values.any()))
    print(largeTable.head())
    print("...")
    print(largeTable.tail())
    return largeTable

In [73]:
#libraries used
import requests
import pandas as pd
import datetime

In [74]:
##api link for get use
##https://documenter.getpostman.com/view/10808728/SzS8rjbc?version=latest

##api link for country codename
##https://documenter.getpostman.com/view/10808728/SzS8rjbc?version=latest#7934d316-f751-4914-9909-39f1901caeb8

##function for pulling new covid cases from the API

def newCasesPerDayFunc(country):
#setting variables for requests
    newCasesPerDay = []
    #country = "japan" ##doesn't work for united-states, united-kingdom,
    url = "https://api.covid19api.com/country/"+country+"/status/confirmed"

    fromdate = "2020-01-01T00:00:00Z"
    todate = "2020-07-13T00:00:00Z"
    payload = {}
    headers= {}
    params = {"from":fromdate, "to":todate}

    #response and json form of response
    response = requests.request("GET", url, headers=headers, data = payload, params=params)
    dataParse = response.json()

    def dailyCasesChange(tdayCases, ydayCases):
        dailyChange = tdayCases - ydayCases
        return dailyChange

    #loop to find data we want
    listLength = len(dataParse)
    for x in range(0, listLength):
        
        #if statement: if not day 1 do the covid cases change calculation
        if x > 0:
            dailyCountryProfile = dataParse[x]
            todaysCases = dailyCountryProfile['Cases']

            ydayCountryProfile = dataParse[x-1]
            ydayCases=ydayCountryProfile['Cases']

            newCasesToday = dailyCasesChange(todaysCases, ydayCases)
            newCasesPerDay.append(newCasesToday)

    return(newCasesPerDay)

In [75]:
startDate = '2020-02-01' #variable for start date of covid cases analysis

#list of all countries daily case data
japanNewCases = newCasesPerDayFunc("japan")
brazilNewCases = newCasesPerDayFunc("brazil")
italyNewCases = newCasesPerDayFunc("italy")
#usaNewCases = newCasesPerDayFunc("")
skNewCases = newCasesPerDayFunc("south-korea")

##Generates a list of all dates
datelist = pd.date_range(startDate, periods=len(japanNewCases)).tolist()

In [76]:
newDateList = []

#loop to change the date formats for table
for item in datelist:
    #print(item)
    #print(type(item))
    newDateFormat = item.strftime("%Y-%m-%d")
    newDateItem = datetime.datetime.strptime(newDateFormat, '%Y-%m-%d').strftime('%Y-%m-%d')
    newDateList.append(newDateItem)
#print(newDateList)

In [77]:
#Create DF based on a dict of lists
#
#
covidCasesDict = {'japanCases':japanNewCases,'brazilCases': brazilNewCases,'italyCases':italyNewCases,'skCases':skNewCases}

casesDF = pd.DataFrame(covidCasesDict, index = pd.to_datetime(newDateList))
casesDF.head()
casesDF.tail()

,japanCases,brazilCases,italyCases,skCases
2020-07-18,356,42619,214,45
2020-07-19,427,45048,276,35
2020-07-20,386,39023,188,44
2020-07-21,411,24831,234,62
2020-07-22,284,20286,169,33


In [78]:
country_etf = bundleStockPrices(['JPXN','EWZ','EWI','EWY'],200)

Any null data: False
               JPXN    EWZ    EWI    EWY
2019-09-30  61.1200  42.13  27.57  56.34
2019-10-01  60.6445  41.68  27.29  56.04
2019-10-02  59.9415  40.78  26.64  55.02
2019-10-03  60.2398  41.20  26.82  55.74
2019-10-04  60.9200  42.11  26.95  56.12
...
             JPXN    EWZ    EWI    EWY
2020-07-08  59.69  30.69  25.10  59.18
2020-07-09  59.35  30.46  24.41  58.82
2020-07-10  60.08  30.79  24.87  58.43
2020-07-13  59.84  30.00  24.61  58.52
2020-07-14  60.34  30.77  25.12  59.05


In [79]:
covidAndStocks = pd.concat([country_etf, casesDF], axis=1, join='inner')
print(covidAndStocks)

               JPXN    EWZ    EWI    EWY  japanCases  brazilCases  italyCases  \
2020-02-03  62.6100  44.43  28.92  58.58           0            0           0   
2020-02-04  63.7300  44.72  29.42  60.24           2            0           0   
2020-02-05  64.2035  44.99  29.83  60.80           0            0           0   
2020-02-06  64.5313  44.17  30.13  61.66           3            0           0   
2020-02-07  64.0138  43.38  29.85  60.45           0            0           0   
2020-02-10  63.9020  42.99  29.88  61.02           5            0           0   
2020-02-11  64.0956  43.91  30.04  61.92           0            0           0   
2020-02-12  63.8922  44.11  30.20  62.90           0            0           0   
2020-02-13  63.3777  43.67  29.94  61.83           2            0           0   
2020-02-14  62.7200  43.86  29.94  62.09           1            0           0   
2020-02-18  61.7694  43.32  30.28  61.08           0            0           0   
2020-02-19  61.8088  43.91  